In [14]:
from dataclasses import dataclass, field
from transformers import HfArgumentParser


@dataclass
class TrainArgs:
    batch_size: int = field(
        default=8, metadata={"help": "Number of batched for training."}
    )


parser = HfArgumentParser(TrainArgs)
parser.print_help()
# In the real world the .parse_args... method would be called with arguments
# to parse data from the commandline.
train_args = parser.parse_args_into_dataclasses(["--batch_size", "4"])
print(train_args)

usage: ipykernel_launcher.py [-h] [--batch_size BATCH_SIZE]

optional arguments:
  -h, --help            show this help message and exit
  --batch_size BATCH_SIZE
                        Number of batched for training. (default: 8)
(TrainArgs(batch_size=4),)


usage: ipykernel_launcher.py [-h] [--batch_size BATCH_SIZE]

optional arguments:
  -h, --help            show this help message and exit
  --batch_size BATCH_SIZE
                        Number of batched for training. (default: 8)
